# Preprocessing

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import GridSearchCV

In [2]:
data = pd.read_csv("res_all_INC.csv")

/usr/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (13,20,21,23,24,26,28,72,73,84,85,86,87,100,101,102) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


On selectionne les lignes pour lesquelles LC n'est pas manquant et RL_STATUT est différent de DD

In [39]:
data1 = data[pd.notna(data['LC']) & (data['RL_STATUT']!='DD')]

On enlève des colonnes

In [40]:
columns_to_exclude = ['T1ID','SPECIESBASAUTHORS','SPECIESAUTHORS','INFRASPECIFICBASAUTHORS','INFRASPECIFICEPITHET','INFRASPECIFICAUTHORS','BASIONYMAUTHORS','DATEP','ACCEPTEDNAMEID','ACCEPTEDNAME','CLIMATE_AQUATIC'] + ['LC','MAJOR_CLIMATE','MAJOR_LIFEFORM','ANID','SUPPRESSED','LIFEFORM_BUDHGHT_MIN', "LIFEFORM_BUDHGHT_MAX", 'RL_NUM', 'RL_STATUT', 'YEAR_ASSESSED', 'LC', 'LCNT', 'RL_V1', 'RL_V2', 'RL_V3', 'RL', 'SONNERAT'] + ['pred_LC', 'p_LC', 'binf.p_LC', 'bsup.p_LC', 'p_rl_V3', 'p_LCNT', 'binf.p_LCNT', 'bsup.p_LCNT', 'miss']

data2 = data1.drop(columns=columns_to_exclude)

In [41]:
categorial_columns = ['CL','N_CLIMATE','CLIMATE_ALTITUDE','CLIMATE_DRY','CLIMATE_TEMPERATE','CLIMATE_TROPICAL','N_LIFEFORM']+['LIFEFORM_CLIMBING','LIFEFORM_DEATH','LIFEFORM_ANNUAL','LIFEFORM_BIENNIAL','LIFEFORM_PERENNIAL','LIFEFORM_GEOPHYTE','LIFEFORM_HYDROPHYTE','LIFEFORM_HELOPHYTE','LIFEFORM_PHANEROPHYTE','LIFEFORM_NANOPHANEROPHYTE','LIFEFORM_HEMICRYPTOPHYTE','LIFEFORM_CHAMAEROPHYTE','LIFEFORM_EPIPHYTE','LIFEFORM_LITHOPHYTE','LIFEFORM_TUBER','LIFEFORM_RHIZOME','LIFEFORM_BULB','LIFEFORM_PSEUDOBULB','LIFEFORM_AQUATIC','LIFEFORM_SUCCULENT','LIFEFORM_CAUDIFORM','LIFEFORM_PARASITE','LIFEFORM_CAUDEX','LIFEFORM_PACHYCAUL','LIFEFORM_EXPOSED']

On marque les variables catégorielles comme telles

In [42]:
data2.replace('?',np.nan,inplace=True)
data2.replace('1',1.,inplace=True)
data2.replace('0',0.,inplace=True)
data2.replace('2',2.,inplace=True)

In [43]:
data2[categorial_columns]=data2[categorial_columns].astype('category')

In [44]:
# conversion object => category
object_columns = (data2.dtypes == 'object')
# index des colonnes categorielles
index_object_columns = object_columns.index[object_columns]
#conversion
data2[index_object_columns]=data2[index_object_columns].astype('category')

In [45]:
# nombres de categories par colonne
data2.describe(exclude=['float64','int64','bool']).loc['unique']

ORDER                           59
SUPER_ORDER                      8
ACCEPTEDFAMILY                 265
FAMILY                         308
FAMILY_CLEAN                   279
FULLNAME                     15824
RANK                             4
GENUS                         3322
SPECIES                       9824
CL                               2
N_CLIMATE                        3
CLIMATE_ALTITUDE                 2
CLIMATE_DRY                      2
CLIMATE_TEMPERATE                2
CLIMATE_TROPICAL                 2
N_LIFEFORM                       4
LIFEFORM_CLIMBING                3
LIFEFORM_DEATH                   3
LIFEFORM_ANNUAL                  2
LIFEFORM_BIENNIAL                2
LIFEFORM_PERENNIAL               2
LIFEFORM_GEOPHYTE                2
LIFEFORM_HYDROPHYTE              2
LIFEFORM_HELOPHYTE               2
LIFEFORM_PHANEROPHYTE            2
LIFEFORM_NANOPHANEROPHYTE        2
LIFEFORM_HEMICRYPTOPHYTE         2
LIFEFORM_CHAMAEROPHYTE           2
LIFEFORM_EPIPHYTE   

On repère les variables catégorielles avec un grand nombre de catégories pour les supprimer

In [46]:
high_cardidality_features_boolean = (data2.describe(exclude=['float64','int64','bool']).loc['unique'] > 100)
high_cardidality_features_list = list(high_cardidality_features_boolean.index[high_cardidality_features_boolean])

In [47]:
X_with_missing_and_categorical = data2.drop(columns=high_cardidality_features_list)
# X_with_missing_and_categorical = data2

In [49]:
y = data1['LC'].astype('int')

On procède au one-hot encoding des variables catégorielles

In [50]:
X_with_missing = pd.get_dummies(X_with_missing_and_categorical)

On impute les données manquantes par un algo 5NN

On normalise les données

In [70]:
from sklearn.preprocessing import StandardScaler
X_with_missing = StandardScaler().fit(X_with_missing).transform(X_with_missing)

/usr/lib/python3.7/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  


In [71]:
from fancyimpute import KNN
X = KNN(k=5).fit_transform(X_with_missing)

Imputing row 1/15824 with 9 missing, elapsed time: 163.661
Imputing row 101/15824 with 31 missing, elapsed time: 163.787


Imputing row 201/15824 with 6 missing, elapsed time: 163.924
Imputing row 301/15824 with 12 missing, elapsed time: 164.041


Imputing row 401/15824 with 31 missing, elapsed time: 164.161
Imputing row 501/15824 with 31 missing, elapsed time: 164.288


Imputing row 601/15824 with 6 missing, elapsed time: 164.431
Imputing row 701/15824 with 25 missing, elapsed time: 164.555


Imputing row 801/15824 with 6 missing, elapsed time: 164.676
Imputing row 901/15824 with 0 missing, elapsed time: 164.785
Imputing row 1001/15824 with 25 missing, elapsed time: 164.865


...

Imputing row 14801/15824 with 10 missing, elapsed time: 180.018
Imputing row 14901/15824 with 35 missing, elapsed time: 180.157


Imputing row 15001/15824 with 35 missing, elapsed time: 180.318
Imputing row 15101/15824 with 16 missing, elapsed time: 180.419
Imputing row 15201/15824 with 6 missing, elapsed time: 180.504


Imputing row 15301/15824 with 31 missing, elapsed time: 180.612
Imputing row 15401/15824 with 12 missing, elapsed time: 180.745


Imputing row 15501/15824 with 31 missing, elapsed time: 180.875
Imputing row 15601/15824 with 31 missing, elapsed time: 181.006


Imputing row 15701/15824 with 31 missing, elapsed time: 181.103
Imputing row 15801/15824 with 9 missing, elapsed time: 181.233


On crée des échantillons de test et d'apprentissage

In [72]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)

# Régression logistique

In [73]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold

## Régularisation l1, paramètre de régularisation choisi par validation croisée

Premier test

In [86]:
LogisticRegression(solver='saga',penalty='l1').fit(X_train,y_train).score(X_test,y_test)

/usr/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.7909504550050556

Fine-tuning

In [87]:
logregcvl1 = LogisticRegressionCV(solver='saga',penalty='l1',Cs=np.logspace(10,20,10),cv=5,n_jobs=-1,verbose=3).fit(X_train,y_train)
print('Train score: ',logregcvl1.score(X_train,y_train))
print('Test score: ',logregcvl1.score(X_test,y_test))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


max_iter reached after 5 seconds


/usr/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


max_iter reached after 5 seconds
max_iter reached after 5 seconds
max_iter reached after 6 seconds
max_iter reached after 6 seconds


max_iter reached after 6 seconds
max_iter reached after 6 seconds


...

max_iter reached after 6 seconds


max_iter reached after 6 seconds


/usr/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.0min finished


max_iter reached after 3 seconds
Train score:  0.8088978766430738
Test score:  0.7939838220424671


/usr/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


## Idem avec régularisation l2

Premier test

In [88]:
LogisticRegression(solver='saga',penalty='l2').fit(X_train,y_train).score(X_test,y_test)

/usr/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.788928210313448

Fine-tuning

In [89]:
logregcvl2 = LogisticRegressionCV(solver='saga',penalty='l2',Cs=np.logspace(5,10,10),cv=5,n_jobs=-1,verbose=3).fit(X_train,y_train)
print('Train score: ',logregcvl2.score(X_train,y_train))
print('Test score: ',logregcvl2.score(X_test,y_test))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


max_iter reached after 10 seconds
max_iter reached after 10 seconds
max_iter reached after 10 seconds
max_iter reached after 10 seconds
max_iter reached after 10 seconds


/usr/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


max_iter reached after 10 seconds
max_iter reached after 10 seconds
max_iter reached after 10 seconds
max_iter reached after 10 seconds
max_iter reached after 10 seconds


max_iter reached after 10 seconds
max_iter reached after 10 seconds
max_iter reached after 10 seconds
max_iter reached after 10 seconds
max_iter reached after 10 seconds


...

max_iter reached after 10 seconds


max_iter reached after 10 seconds
max_iter reached after 10 seconds
max_iter reached after 10 seconds
max_iter reached after 10 seconds
max_iter reached after 10 seconds


/usr/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  1.7min remaining:  2.5min
/usr/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.7min finished


max_iter reached after 2 seconds
Train score:  0.8087293562521065
Test score:  0.7934782608695652


/usr/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


# SVM

In [90]:
from sklearn.svm import LinearSVC, SVC

## Linéaire

Premiers essais

In [91]:
print(LinearSVC(penalty='l1',dual=False).fit(X_train,y_train).score(X_test,y_test))

0.7934782608695652


/usr/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [92]:
print(LinearSVC(penalty='l2').fit(X_train,y_train).score(X_test,y_test))

0.788928210313448


/usr/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Fine-tuning

In [93]:
svclinl1 = GridSearchCV(LinearSVC(penalty='l1',dual=False), param_grid={'C':np.logspace(-1,4,10)}, cv=5,n_jobs=-1,verbose=3).fit(X_train,y_train)
print('Train score:',svclinl1.score(X_train,y_train))
print('Test score:',svclinl1.score(X_test,y_test))

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   11.6s


[Parallel(n_jobs=-1)]: Done  44 out of  50 | elapsed:   51.6s remaining:    7.0s


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   56.4s finished


Train score: 0.8088136164475902
Test score: 0.7937310414560161


/usr/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [94]:
svclinl2 = GridSearchCV(LinearSVC(penalty='l2'), param_grid={'C':np.logspace(-5,0,10)}, cv=5,n_jobs=-1,verbose=3).fit(X_train,y_train)
print('Train score:',svclinl2.score(X_train,y_train))
print('Test score:',svclinl2.score(X_test,y_test))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.5s


[Parallel(n_jobs=-1)]: Done  44 out of  50 | elapsed:   25.7s remaining:    3.5s


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   27.1s finished


Train score: 0.8090663970340412
Test score: 0.7937310414560161


/usr/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


## Polynômial de degré 3

In [95]:
svcpoly = GridSearchCV(SVC(kernel='poly',degree=3,gamma='scale',coef0=1), param_grid={'C':np.logspace(-2,2,5)},n_jobs=-1,verbose=3,cv=3).fit(X_train,y_train)
print('Train score:',svcpoly.score(X_train,y_train))
print('Test score:',svcpoly.score(X_test,y_test))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Done   4 out of  15 | elapsed:   41.5s remaining:  1.9min


[Parallel(n_jobs=-1)]: Done  10 out of  15 | elapsed:   47.0s remaining:   23.5s


[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  1.0min finished


Train score: 0.8394000674081564


Test score: 0.7934782608695652


# Random Forests

In [96]:
from sklearn.ensemble import RandomForestClassifier

Premier essai

In [97]:
print(RandomForestClassifier().fit(X_train,y_train).score(X_test,y_test))

0.7798281092012134


/usr/lib/python3.7/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Fine-tuning

In [98]:
rf = GridSearchCV(RandomForestClassifier(n_estimators=100), param_grid={'criterion':['gini','entropy'], 'max_depth':[10,20,50], 'max_features':[10,20,40]},n_jobs=-1,verbose=3,cv=5).fit(X_train,y_train)
print('Training score: ',rf.score(X_train,y_train))
print("Test score: ",rf.score(X_test,y_test))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    2.9s


[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:   34.8s finished


Training score:  0.9657903606336367
Test score:  0.7980283114256825


# Gradient Boosting

Premier essai

In [99]:
from sklearn.ensemble import GradientBoostingClassifier
print(GradientBoostingClassifier().fit(X_train,y_train).score(X_test,y_test))

0.8008088978766431


Fine-tuning

In [100]:
gb = GridSearchCV(GradientBoostingClassifier(n_estimators=100,max_depth=20),param_grid={'loss':['deviance','exponential'], 'learning_rate':np.logspace(-2,2,5)},n_jobs=-1,verbose=3,cv=5).fit(X_train,y_train)
print('Train score:',gb.score(X_train,y_train))
print('Test score:',gb.score(X_test,y_test))

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  3.2min


[Parallel(n_jobs=-1)]: Done  44 out of  50 | elapsed: 10.6min remaining:  1.4min


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 11.3min finished


Train score: 0.9825581395348837
Test score: 0.7874115267947421


# Extreme gradient boosting

In [101]:
import xgboost as xgb

Premier essai

In [102]:
print(xgb.XGBClassifier().fit(X_train,y_train).score(X_test,y_test))

0.7977755308392316


Fine-tuning

In [103]:
xgb_model = GridSearchCV(xgb.XGBClassifier(n_estimators=400,n_jobs=-1),param_grid={'booster':['gbtree','gblinear','dart']},n_jobs=-1,verbose=3,cv=5).fit(X_train,y_train)
print('Train score:',xgb_model.score(X_train,y_train))
print('Test score:',xgb_model.score(X_test,y_test))

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


[Parallel(n_jobs=-1)]: Done   4 out of  15 | elapsed:   12.3s remaining:   33.7s


[Parallel(n_jobs=-1)]: Done  10 out of  15 | elapsed:   17.5s remaining:    8.8s


[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   41.1s finished


Train score: 0.8207785642062689
Test score: 0.7975227502527806


# Extra trees

In [104]:
from sklearn.ensemble import ExtraTreesClassifier

Premier essai

In [105]:
print(ExtraTreesClassifier().fit(X_train,y_train).score(X_test,y_test))

0.7798281092012134


/usr/lib/python3.7/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Fine-tuning

In [106]:
xt = GridSearchCV(ExtraTreesClassifier(n_estimators=500,n_jobs=-1),param_grid={'criterion':['gini','entropy'],'max_depth':[3,10,20,50],'max_features':[3,10,20,50]},n_jobs=-1,verbose=3,cv=5).fit(X_train,y_train)
print('Train score:',xt.score(X_train,y_train))
print('Test score:',xt.score(X_test,y_test))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    7.1s


[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:  2.7min


[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  4.6min finished


Train score: 0.9609032692955848
Test score: 0.7990394337714863


# LightGBM

In [107]:
import lightgbm as lgb
lgb_model = lgb.LGBMClassifier(n_estimators=100,n_jobs=-1).fit(X_train,y_train)
lgb_model.score(X_test,y_test)

0.8063700707785642